In [23]:
import numpy as np
import pandas as pd
import math
from operator import le

In [24]:
entrada='''Me gustas cuando callas porque estás como ausente,
y me oyes desde lejos, y mi voz no te toca.
Parece que los ojos se te hubieran volado
y parece que un beso te cerrara la boca.

Como todas las cosas están llenas de mi alma
emerges de las cosas, llena del alma mía.
Mariposa de sueño, te pareces a mi alma,
y te pareces a la palabra melancolía.

Me gustas cuando callas y estás como distante.
Y estás como quejándote, mariposa en arrullo.
Y me oyes desde lejos, y mi voz no te alcanza:
déjame que me calle con el silencio tuyo.

Déjame que te hable también con tu silencio
claro como una lámpara, simple como un anillo.
Eres como la noche, callada y constelada.
Tu silencio es de estrella, tan lejano y sencillo.

Me gustas cuando callas porque estás como ausente.
Distante y dolorosa como si hubieras muerto.
Una palabra entonces, una sonrisa bastan.
Y estoy alegre, alegre de que no sea cierto.
'''

In [25]:
print(f'\t-----CODIFICACIÓN LZW-----\n')
# Inicializa diccionario estatico, con las letras ordenadas
diccionario_estatico = dict.fromkeys(sorted(entrada))
# inicializo diccionario estatico con los indices y valores 
diccionario_estatico.update(zip(diccionario_estatico,[format(x,'b') for x in range(len(diccionario_estatico))]))
# Inicializo dataframe dinamico -> Copia diccionario estatico
df = pd.DataFrame([[key, int(diccionario_estatico[key],2), diccionario_estatico[key]] for key in diccionario_estatico.keys()], columns=['letra', 'indice_anterior','indice_binario'])
largo_indice= math.ceil(np.log2(len(df)))  # el largo del bit de cada palabra que se va codificando 
indice_dic_dinamico = len(df)
# Acumulador de letras
acumulador = 0
# indice que recorre el texto, la longitud del texto
indice = 0
# Valor que lleva la codificacion
codificacion = ''
# Ciclo que llena el dataframe con la codificacion de los simbolos
while indice + acumulador < len(entrada):
    # LetraCodigo -> Es  la letra o palabra formada para la codificacion del texto, en caso de entrar en el primer if
    letra_codigo =  entrada[indice:indice+acumulador+1]
    # Letras -> Es la letra que se va comprobando si está en el dinamico o si no está para agregarla
    letra = entrada[indice:indice+acumulador+2]
    if letra not in df.values:
        # se suma 0.5 esto por que por ejemplo si el indice es 64, necesita minimo 7 bit para representarse
        largo_indice = math.ceil(np.log2(indice_dic_dinamico+0.5))
        # Codifica el mensaje añadiendo los bits al inicio y la codificacion del indice de la ultima letra o letras 
        dato_codificado = ('{0:0'+str(largo_indice)+'b}').format(df[df['letra']==letra_codigo].index.values[0])
        # Se agrega el dato al dataframe [letra, valor codificado]
        df.loc[len(df)] = [letra, df[df['letra']==letra_codigo].index.values[0],dato_codificado]
        #Indice -> es con el que accedo a los valores del texto, para obtener las palabras de ahi 
        indice += acumulador + 1
        # indiceDiccionarioDinamico el encargado para calcular la longitud de bits de cada palabra-simbolo-letra codificada
        indice_dic_dinamico += 1
        # acumulador  se reinicia para coger empezar a coger al nueva palabra
        acumulador = 0
        # Se suma la codificacion
        codificacion += dato_codificado
    else:
        # el acomulador se aumenta para ir concatenando en caso de que ya se encuentre la palabra en la lista
        acumulador += 1
# Se agrega el ultimo dato
dato_codificado = ('{0:0'+str(largo_indice)+'b}').format(df[df['letra']==letra_codigo].index.values[0])
df.loc[len(df)] = [letra,df[df['letra']==letra_codigo].index.values[0],dato_codificado]
codificacion += dato_codificado
# Se crea un csv con el dataset
df.to_csv('dataset.csv')

print(f'\n\t\tmensaje codificado:\n {codificacion}')
f = open ('mensaje_codificado.txt','w')
f.write(codificacion)
f.close()
df


	-----CODIFICACIÓN LZW-----


		mensaje codificado:
 00100001000100000101001001111101110101111000110101110100000100111101111100110101100001000001100111000000110101011001011010111000000101101001100101110000110110011111010100000100010101100010001101011110001111001100100101110110110000110101010110010001001100000111100010001000001000000000100001000000100101110101000001100101000011000011000000100100001000011101101100000010010110001000100101010011001001110100000100000001101001100101110010100000000101000000011001010001000000010011000011011010011110000001001111000110010001111000110100000110000000000100100011010011100001000100011110101000100000001010000001011001100010000000010001100101100001001011110001110100101000011100000001001100011111000011100001010000010001000111000011001101101001000110010011101000110101010100100011110001111001011110110111110101000001000000010001111100011000000000010000111001000011011011110010100001111100000111001000111010001110010111100000110110011101011100110111010100000

,letra,indice_anterior,indice_binario
0,\n,0,0
1,,1,1
2,",",2,10
3,.,3,11
4,:,4,100
...,...,...,...
517,a c,167,0010100111
518,cie,345,0101011001
519,ert,141,0010001101
520,to.\n,479,0111011111


In [26]:
############# -DECODIFICACION- ############

f = open ('mensaje_a_decodificar.txt','r')
textoLLega = f.read()
# se cambian los valores del diccionario dinamico , las claves por los valores, para hacer un manejo más eficiente
diccionarioDinamicoDeco = {}
for key,val in diccionario_estatico.items():
    diccionarioDinamicoDeco[int(val,2)] = key

# DECODOFICACION COMPLETA
indice  = 0  # indice que recorre el texto, la longitud del texto de salida
acomulador = 0 # para ir aumentando el paso a traves del texto
indiceDiccDinamico =  len(diccionario_estatico)  # inidice donde empieza a agregar elementos al diccionario
textoDecodificado = ""  # Texto que se va construyendo a medida que llegan los bits
longitudTextodeco = 0  #longitus inicial del texto decodificado, que se usa para validar y agregarle informacion 

In [27]:
print(f'\t-----DECODIFICACIÓN LZW-----\n')
#funcion que me va a reconstruir el diccionario dinamico del receptor al tiempo que me sigue completando el texto de salida
#esta funcion recorre el texto que se ha ido decodificando , desde el ultimo indice en que quedo y recontruye diccionario, para luego seguir comprobando bits que llegan si ya estan en el o si toca de nuevo poner el diccionario a agregar mas palabras
def llenarDic():
    # defino las variables que uso que son globales en la funcion 
    global textoDecodificado
    global indice
    global acomulador
    global indiceDiccDinamico
    while indice + acomulador  < longitudTextodeco:
        # Letras -> Es la letra que se va comprobando si está en el dinamico o si no está para agregarla
        Letras =  textoDecodificado[indice: indice+acomulador+2]   # ejemplo -> entrada= "hola"  si i = 0 y acomulador = 0 , entrada[i: i+acomulador+2] retorna "ho"
        # largo_indice, lleva el tamaño de cada palabra codificada
        if Letras not in diccionarioDinamicoDeco.values():
            # diccionadioDinamico que queda de la forma   {"letra": (indice, binario)}
            diccionarioDinamicoDeco[indiceDiccDinamico]  = (Letras)  
            #diccionarioDinamicoDecoo[Letras] = indiceDiccDinamico
            # aumento de las variables de ayuda ----------------------//////
            # indiceDiccionarioDinamico el encargado para calcular la longitud de bits de cada palabra-simbolo-letra codificada
            indiceDiccDinamico +=1
            #Indice -> es con el que accedo a los valores del texto , para obtener las palabras de ahi 
            indice += acomulador + 1
            # acomulador  se reinicia para coger empezar a coger al nueva palabra
            acomulador = 0
            if(indiceVal in diccionarioDinamicoDeco.keys() and len(textoDecodificado) == longitudTextodeco):
                # aqui pruebo que si se agrego una nueva palabra al diccionario y esta va en la posicion en la que iba
                # el texto decodificado, se la agredo al texto decodificado
                textoDecodificado+= diccionarioDinamicoDeco[indiceVal]
        else:
            # el acomulador se aumenta para ir concatenando en caso de que ya se encuentre la palabra en la lista
            acomulador += 1

# FUNCION QUE ME VA CONTRUYENDO EL TEXTO DE SALIDA Y ME VA CREANDO EL DICCIONARIO DINAMICO AL MISMO TIEMPO
longitudBits =  math.ceil(np.log2(indiceDiccDinamico+0.5)) #se toma en cuanto se debe iniciar a coger los bits que llegan 
posicionBit = 0
conteoPalabras = indiceDiccDinamico # esta me ayuda a ver en que posicion del diccionario voy y comprobar la longitud de bits

while posicionBit in range(len(textoLLega)):

    porcion =  textoLLega[posicionBit:posicionBit+longitudBits]
    conteoPalabras += 1
    indiceVal =  int(porcion, 2) # pasa lo que llega a entero 

    # SI NO ESTÁ EL BINARIO QUE LLEGO, PONGO A RECONSTRUIR DICCIONARIO CON EL TEXTO QUE HASTA EL MOMENTO HA SIDO DECODIFICADO
    if indiceVal not in diccionarioDinamicoDeco:
        # TENGO EN CUENTA LA LONGITUD DEL TEXTO EN ESTE MOMENTO PARA RECORRELO EN LA FUNCION
        longitudTextodeco =  len(textoDecodificado)
        llenarDic()

    else:
        #AGREGO AL TEXTO DE SALIDA LO QUE YA TENGA EL DICC DINAMICO HASTA EL MOMENTO
        textoDecodificado += diccionarioDinamicoDeco[indiceVal]
       

    # SI YA ES EL ULTIMO i, Vuelvo y llamo por ultima ver el metodo con la longitud del texto final, para terminar de reconstruit el texto dinamico 
    if(posicionBit+longitudBits not in range(len(textoLLega))):
        longitudTextodeco =  len(textoDecodificado)-1
        llenarDic()

    posicionBit += longitudBits
    longitudBits =  math.ceil(np.log2(conteoPalabras+0.5))

print(textoDecodificado)

	-----DECODIFICACIÓN LZW-----

Me gustas cuando callas porque estás como ausente,
y me oyes desde lejos, y mi voz no te toca.
Parece que los ojos se te hubieran volado
y parece que un beso te cerrara la boca.

Como todas las cosas están llenas de mi alma
emerges de las cosas, llena del alma mía.
Mariposa de sueño, te pareces a mi alma,
y te pareces a la palabra melancolía.

Me gustas cuando callas y estás como distante.
Y estás como quejándote, mariposa en arrullo.
Y me oyes desde lejos, y mi voz no te alcanza:
déjame que me calle con el silencio tuyo.

Déjame que te hable también con tu silencio
claro como una lámpara, simple como un anillo.
Eres como la noche, callada y constelada.
Tu silencio es de estrella, tan lejano y sencillo.

Me gustas cuando callas porque estás como ausente.
Distante y dolorosa como si hubieras muerto.
Una palabra entonces, una sonrisa bastan.
Y estoy alegre, alegre de que no sea cierto.


